<a href="https://colab.research.google.com/github/ahatasham5/NewspaperSummarizationLLM/blob/main/Newspaper_Sum_with_gemini_1_5_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [2]:

# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatibl

In [3]:

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

In [5]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
100% 503M/503M [00:24<00:00, 24.2MB/s]
100% 503M/503M [00:24<00:00, 21.4MB/s]


In [6]:
# Unzip the downloaded file
!unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [7]:
# prompt: i want to load this data set

import pandas as pd

# Specify the path to the extracted directory
data_dir = "/content/cnn_dailymail"

# Load the training data
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))

# Load the validation data
val_df = pd.read_csv(os.path.join(data_dir, "validation.csv"))

# Load the test data
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))


In [8]:
!pip install -q -U google-generativeai

In [9]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
def to_markdown(text):
  text = text.replace('*', '\\*')
  return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [10]:
from google.colab import userdata

In [11]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [18]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [19]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [20]:
import ast

def preprocess_data(df):
    data = []
    for _, row in df.iterrows():
        text = row['article']
        summary = row['highlights']
        data.append({'text': text, 'summary': summary})
    return data

train_data = preprocess_data(train_df)
val_data = preprocess_data(val_df)

In [31]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 20.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [32]:
from peft import LoraConfig

In [33]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [42]:
from google.generativeai import GenerativeModel

In [45]:
!pip show peft

Name: peft
Version: 0.10.0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [46]:
!pip uninstall peft

Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/peft-0.10.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/peft/*
Proceed (Y/n)? Y
  Successfully uninstalled peft-0.10.0


In [47]:
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-brig81gq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-brig81gq
  Resolved https://github.com/huggingface/peft.git to commit 32f38788705b15f3aa718057a8f361969e63d273
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.10.1.dev0-py3-none-any.whl size=249079 sha256=a9ec32c96a4e801371abaed29c1afee5e120c0d0a140ea8588d4727f7213a2c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-6ueh4x3n/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft


In [1]:
from google.generativeai import GenerativeModel
from peft import get_peft_trainer, LoraConfig

# Load the Gemini-1.5-pro model
model = GenerativeModel('gemini-1.5-pro-latest')

# Extract the underlying model
underlying_model = model.model

# Set up the LoRA configuration
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj",
                    "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# Create the PEFT trainer
trainer = get_peft_trainer(underlying_model, lora_config)

# Set the training parameters
trainer.train(
    train_data=train_data,
    val_data=val_data,
    max_epochs=10,
    batch_size=8,
    learning_rate=1e-4,
    checkpoint_dir="checkpoint_dir"
)

ImportError: cannot import name 'get_peft_trainer' from 'peft' (/usr/local/lib/python3.10/dist-packages/peft/__init__.py)